In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


# Cases, fine age split

In [47]:

firstDate = np.datetime64('2021-08-01')

latestsubdir = list(os.walk(path_dash))[0][1][-1]
latestdir = path_dash + latestsubdir
latestdir

dfAge = pd.read_csv(latestdir+'\\Regionalt_DB\\18_fnkt_alder_uge_testede_positive_nyindlagte.csv',delimiter=';',encoding='latin1')
dfAge['Nyindlagte pr. 100.000 borgere'] = pd.to_numeric(dfAge['Nyindlagte pr. 100.000 borgere'].str.replace(',','.'))

dfAge.tail(18)


,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive
823,2021-W46,0-2,5615,200.0,10.9,10300,366.0
824,2021-W46,3-5,20924,526.0,1.6,39163,985.0
825,2021-W46,6-11,34985,1561.0,4.1,128809,5748.0
826,2021-W46,12-15,26072,550.0,0.7,71065,1498.0
827,2021-W46,16-19,16120,406.0,0.7,44169,1112.0
828,2021-W46,20-39,19547,487.0,5.9,289574,7220.0
829,2021-W46,40-64,24023,438.0,7.8,454142,8287.0
830,2021-W46,65-79,10908,253.0,15.8,98506,2281.0
831,2021-W46,80+,6295,154.0,43.2,18380,451.0
832,2021-W47,0-2,5826,203.0,8.7,10687,373.0


In [3]:
curYears = dfAge.Uge.apply(lambda x: int(x[:4]))

# Remove everything before 2021
curdf = dfAge[curYears > 2020].copy()


weekNums = curdf.Uge.apply(lambda x: int(x[-2:]))


# Uge 36: 07-09-2021 (tirsdag) og 12-09-2021 (søndag)

weekOffset = weekNums - 36
# allDates = weekOffset.apply(lambda x:np.datetime64('2021-09-07') + np.timedelta64(x*7,'D'))
allDates = weekOffset.apply(lambda x:np.datetime64('2021-09-12') + np.timedelta64(x*7,'D'))

allDates
curdf['Dato'] = allDates

In [53]:


# fig,ax1 = plt.subplots()
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal positive'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal positive'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)
    toAdd = thisdf['Antal positive'].values
    toAdd = thisdf['Antal positive'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,label=curAge)
    ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Nye smittetilfælde\n[ugentligt gennemsnit, per dag]')
ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_xlim(left=firstDate,right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_Cases')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:


# fig,ax1 = plt.subplots()
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal positive'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal positive'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)
    toAdd = thisdf['Antal positive'].values
    # toAdd = thisdf['Antal positive'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentligt nye smittetilfælde')
ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_xlim(left=firstDate,right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_CasesWeekly')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:


fig,ax1 = plt.subplots()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
# ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal positive'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal positive'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('tab20b',len(allAgeGroups)+1)
# cmap = plt.cm.get_cmap('inferno',len(allAgeGroups)+5)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal positive'],label=curAge)
    toAdd = thisdf['Positive pr. 100.000 borgere'].values
    # toAdd = thisdf['Antal positive'].values/7
    # curCount = curCount + toAdd
    ax1.plot(thisdf.Dato,toAdd,'.-',color=cmap(len(allAgeGroups)-counter),label=curAge)
    # ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # # ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    # ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentligt nye smittetilfælde per 100.000')
# ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_xlim(left=firstDate,right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
# ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
# ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_CasesWeeklyPer100000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Testede

In [7]:
curdf.tail()

,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Dato
836,2021-W47,16-19,17189,404.0,1.5,47098,1106.0,2021-11-28
837,2021-W47,20-39,20799,479.0,7.6,308115,7095.0,2021-11-28
838,2021-W47,40-64,26208,459.0,7.9,495447,8670.0,2021-11-28
839,2021-W47,65-79,11499,274.0,17.1,103843,2478.0,2021-11-28
840,2021-W47,80+,6533,139.0,34.6,19076,406.0,2021-11-28


In [50]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal testede'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Antal testede'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Antal testede'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal testede'],label=curAge)
    toAdd = thisdf['Antal testede'].values
    toAdd = thisdf['Antal testede'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,label=curAge)
    ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Testede \n[ugentligt gennemsnit, per dag]')
ax2.set_ylabel('Andel af alle testede')


ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_xlim(left=firstDate,right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_Testede')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
curdf.tail()

,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Dato
836,2021-W47,16-19,17189,404.0,1.5,47098,1106.0,2021-11-28
837,2021-W47,20-39,20799,479.0,7.6,308115,7095.0,2021-11-28
838,2021-W47,40-64,26208,459.0,7.9,495447,8670.0,2021-11-28
839,2021-W47,65-79,11499,274.0,17.1,103843,2478.0,2021-11-28
840,2021-W47,80+,6533,139.0,34.6,19076,406.0,2021-11-28


In [49]:


fig,ax1 = plt.subplots()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
# ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Testede pr. 100.000 borgere'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Testede pr. 100.000 borgere'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Testede pr. 100.000 borgere'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('tab20b',len(allAgeGroups)+1)
# cmap = plt.cm.get_cmap('inferno',len(allAgeGroups)+5)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Antal Testede'],label=curAge)
    toAdd = thisdf['Testede pr. 100.000 borgere'].values
    # toAdd = thisdf['Antal Testede'].values/7
    # curCount = curCount + toAdd
    ax1.plot(thisdf.Dato,toAdd,'.-',color=cmap(len(allAgeGroups)-counter),label=curAge)
    # ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # # ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    # ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentligt testede per 100.000')
# ax2.set_ylabel('Andel af tilfælde')

# ax1.plot(thisdf.Dato,allSums,'k:')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_xlim(left=firstDate,right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
# ax1.set_ylim(top = 25000)
# ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
# ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_TestedeWeeklyPer100000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
thisdf.columns

Index(['Uge', 'Aldersgruppe', 'Testede pr. 100.000 borgere',
       'Positive pr. 100.000 borgere', 'Nyindlagte pr. 100.000 borgere',
       'Antal testede', 'Antal positive', 'Dato'],
      dtype='object')

In [48]:


fig,ax1 = plt.subplots()
fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Antal testede'].values.shape)

prevCount = curCount

allSums = 100*curdf.groupby('Dato')['Antal positive'].sum().values/curdf.groupby('Dato')['Antal testede'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('tab20b',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    
    toAdd = 100*thisdf['Antal positive'].values/thisdf['Antal testede'].values 
    
    ax1.plot(thisdf.Dato,toAdd,'.-',color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')

ax1.set_ylabel('Positive percentage [%]')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_xlim(left=firstDate,right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
# ax1.set_ylim(top = 25000)
ax1.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_PosPct')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Indlagte

In [11]:
print(curdf.tail(9).sum())
display(curdf.tail(9))
print((6324+25698+ 88230)/831807)
print((189.0+522.0+ 3459.0)/15867.0)
# Testede pr. 100.000 borgere

Uge                               2021-W472021-W472021-W472021-W472021-W472021-W...
Aldersgruppe                                 0-23-56-1112-1516-1920-3940-6465-7980+
Testede pr. 100.000 borgere                                                  171975
Positive pr. 100.000 borgere                                                 4725.0
Nyindlagte pr. 100.000 borgere                                                 83.0
Antal testede                                                               1228479
Antal positive                                                              28790.0
dtype: object


,Uge,Aldersgruppe,Testede pr. 100.000 borgere,Positive pr. 100.000 borgere,Nyindlagte pr. 100.000 borgere,Antal testede,Antal positive,Dato
832,2021-W47,0-2,5826,203.0,8.7,10687,373.0,2021-11-28
833,2021-W47,3-5,22243,558.0,2.1,41632,1045.0,2021-11-28
834,2021-W47,6-11,36048,1670.0,2.4,132721,6149.0,2021-11-28
835,2021-W47,12-15,25630,539.0,1.1,69860,1468.0,2021-11-28
836,2021-W47,16-19,17189,404.0,1.5,47098,1106.0,2021-11-28
837,2021-W47,20-39,20799,479.0,7.6,308115,7095.0,2021-11-28
838,2021-W47,40-64,26208,459.0,7.9,495447,8670.0,2021-11-28
839,2021-W47,65-79,11499,274.0,17.1,103843,2478.0,2021-11-28
840,2021-W47,80+,6533,139.0,34.6,19076,406.0,2021-11-28


0.14456718926385567
0.26280960484023447


In [37]:
# curdf['Nyindlagte pr. 100.000 borgere']
# curdf

In [12]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Nyindlagte pr. 100.000 borgere'].values.shape)
# ax1.plot(thisdf.Dato,thisdf['Nyindlagte pr. 100.000 borgere'],label=curAge)

prevCount = curCount

allSums = curdf.groupby('Dato')['Nyindlagte pr. 100.000 borgere'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('Spectral',len(allAgeGroups)+1)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    # ax1.plot(thisdf.Dato,thisdf['Nyindlagte pr. 100.000 borgere'],label=curAge)
    toAdd = thisdf['Nyindlagte pr. 100.000 borgere'].values
    toAdd = thisdf['Nyindlagte pr. 100.000 borgere'].values/7
    curCount = curCount + toAdd
    # ax1.plot(thisdf.Dato,curCount,label=curAge)
    ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,label=curAge)
    ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Nyindlæggelser \n[ugentligt gennemsnit, per dag]')
ax2.set_ylabel('Andel af alle nyindlagte')


ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
ax1.set_xlim(left=firstDate,right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=1)
ax1.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_NyindlagtePer100000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
curdf.columns

Index(['Uge', 'Aldersgruppe', 'Testede pr. 100.000 borgere',
       'Positive pr. 100.000 borgere', 'Nyindlagte pr. 100.000 borgere',
       'Antal testede', 'Antal positive', 'Dato'],
      dtype='object')

In [46]:


fig,ax1 = plt.subplots()
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
# ax2.set_facecolor('lightgrey')


curAge = '6-11'

thisdf = curdf[curdf.Aldersgruppe == curAge]
curCount = np.zeros(thisdf['Nyindlagte pr. 100.000 borgere'].values.shape)

prevCount = curCount

allSums = 100*curdf.groupby('Dato')['Nyindlagte pr. 100.000 borgere'].sum().values

allAgeGroups = curdf.Aldersgruppe.unique()

cmap = plt.cm.get_cmap('tab20b',len(allAgeGroups)+1)
# cmap = plt.cm.get_cmap('inferno',len(allAgeGroups)+5)

counter=0
for curAge in allAgeGroups:
    thisdf = curdf[curdf.Aldersgruppe == curAge]
    toAdd = 100*thisdf['Nyindlagte pr. 100.000 borgere'].values
    # toAdd = thisdf['Antal Testede'].values/7
    # curCount = curCount + toAdd
    ax1.plot(thisdf.Dato,toAdd,'.-',color=cmap(len(allAgeGroups)-counter),label=curAge)
    # ax1.fill_between(thisdf.Dato,curCount,prevCount,color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # # ax2.fill_between(thisdf.Dato,curCount/allSums,prevCount/allSums,color=cmap(len(allAgeGroups)-counter),label=curAge)
    # # ax2.fill_between(thisdf.Dato,curCount/(allSums/7),prevCount/(allSums/7),color=cmap(len(allAgeGroups)-counter),label=curAge)
    
    # prevCount = curCount
    counter = counter+1
    
    
# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    # ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=1,facecolor='lightgrey',alpha=0.25,label=int(k==0)*'Weekend')

ax1.set_ylabel('Ugentlige nyindlæggelser per 100.000')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_xlim(left=firstDate,right=thisdf.Dato.max())
ax1.set_ylim(bottom=0)
ax1.set_ylim(top = 5000)
ax1.legend(loc='upper left',fontsize=12)
# ax2.legend(loc='upper left',fontsize=12)
ax1.grid(axis='y')
# ax2.grid(axis='y')
fig.tight_layout()


fig.savefig(path_figs+'Agedistribution_AdmissionsWeeklyPer100000')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …